In [64]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

# Build paths
# source_path = Path(DOWNLOAD_DIR) / f'df_finviz_{date_str}.pkl'
# dest_path = Path(DEST_DIR) / 'df_finviz.pkl'
source_path = Path(DOWNLOAD_DIR) / f'df_finviz_{date_str}.parquet'
dest_path = Path(DEST_DIR) / 'df_finviz.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

# Example file operations:
# import pandas as pd
# df = pd.read_pickle(source_path)
# df.to_pickle(dest_path)

source_path: C:\Users\ping\Downloads\df_finviz_2025-04-01.parquet
dest_path: ..\data\df_finviz.parquet


In [65]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell

In [66]:
df = pd.read_parquet(source_path, engine='pyarrow')
# df = pd.read_pickle(source_path)
display(df.info(), df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   No.              1560 non-null   object
 1   Ticker           1560 non-null   object
 2   Company          1560 non-null   object
 3   Sector           1560 non-null   object
 4   Industry         1560 non-null   object
 5   Market Cap       1560 non-null   object
 6   Dividend         1560 non-null   object
 7   Perf Week        1560 non-null   object
 8   Perf Month       1560 non-null   object
 9   Perf Quart       1560 non-null   object
 10  Perf Half        1560 non-null   object
 11  Perf Year        1560 non-null   object
 12  Perf YTD         1560 non-null   object
 13  Beta             1560 non-null   object
 14  ATR              1560 non-null   object
 15  Volatility W     1560 non-null   object
 16  Volatility M     1560 non-null   object
 17  SMA20            1560 non-null   

None

,No.,Ticker,Company,Sector,Industry,Market Cap,Dividend,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,All-Time High,All-Time Low,RSI,Gap,Avg Volume,Rel Volume,Volume,Price,Change,Single Category,Asset Type,AUM
0,101,ADP,Automatic Data Processing Inc,Technology,Software - Application,124.72B,1.86%,2.39%,-3.80%,3.50%,12.34%,22.74%,4.72%,0.75,5.49,2.04%,2.02%,2.28%,0.83%,8.06%,-5.05%,6.00%,-5.05%,32.55%,-5.05%,20998.85%,55.78,-0.05%,1.81M,0.66,"1,198,981",306.54,0.33%,-,-,-
1,102,FI,"Fiserv, Inc",Technology,Information Technology Services,124.61B,-,-0.27%,-6.64%,6.76%,24.46%,38.91%,8.08%,0.92,5.08,2.32%,2.47%,1.68%,-0.02%,14.96%,-6.95%,8.45%,-6.95%,52.08%,-6.95%,89814.05%,51.93,-0.21%,3.03M,1.11,"3,370,446",222.01,0.53%,-,-,-
2,103,VRTX,"Vertex Pharmaceuticals, Inc",Healthcare,Biotechnology,124.48B,-,-4.77%,0.30%,18.59%,4.72%,15.96%,20.37%,0.51,12.70,2.42%,2.63%,-3.00%,1.36%,3.07%,-6.72%,15.62%,-6.76%,28.29%,-6.76%,14815.08%,44.78,0.30%,1.47M,0.85,"1,248,959",484.74,-0.02%,-,-,-
3,104,BHP,BHP Group Limited ADR,Basic Materials,Other Industrial Metals & Mining,123.72B,4.46%,-1.75%,-0.12%,-0.87%,-21.34%,-15.44%,-0.10%,0.97,0.95,1.60%,1.66%,-1.62%,-2.27%,-8.61%,-7.12%,3.13%,-22.83%,3.13%,-42.45%,6321.59%,43.24,0.10%,2.60M,0.97,"2,513,260",48.78,0.49%,-,-,-
4,105,BMY,Bristol-Myers Squibb Co,Healthcare,Drug Manufacturers - General,121.17B,4.15%,0.61%,-0.53%,3.24%,16.97%,9.81%,5.29%,0.41,1.57,2.70%,2.62%,-1.18%,1.37%,12.44%,-5.97%,11.23%,-5.97%,51.33%,-26.87%,1350.83%,50.01,-0.10%,11.68M,0.82,"9,613,249",59.55,-2.36%,-,-,-


In [67]:
import numpy as np

def convert_B_M_K_to_million(value_str):
  """
  Convert financial string values with suffixes to numeric values in billions
  Examples:
  '104.27B' -> 104,270
  '104.27M' -> 104.27
  '104.27K' -> 0.10427
  '-' -> np.nan
  """
  
  if not isinstance(value_str, str):
    return value_str  # Return the original value if not a string
    
  if value_str == '-':
    return np.nan
    
  # Remove any whitespace
  value_str = value_str.strip()
  
  # Handle suffixes
  multipliers = {
    'B': 1000,  
    'M': 1,
    'K': 0.001
  }
  
  try:
    # Extract the numeric part and suffix
    if value_str[-1] in multipliers:
      number = float(value_str[:-1])
      multiplier = multipliers[value_str[-1]]
      return number * multiplier
    else:
      # If no suffix, try to convert directly to float
      return float(value_str)
  except (ValueError, IndexError):
    return np.nan

# Example usage:
# df['Market Cap Numeric'] = df['Market Cap'].apply(convert_B_M_K_to_million)

In [68]:
import pandas as pd
import numpy as np

def process_percentage_columns(df):
    """
    Identifies and processes columns in a DataFrame where values end with '%'.
    The function cleans, converts to numeric, renames, and prints which columns were modified.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The modified DataFrame with processed percentage columns.
                      Returns the original DataFrame if no percentage columns are found.
    """

    # Identify columns where values END WITH '%'
    percent_cols = [
        col for col in df.columns
        if df[col].dtype == 'object'
        and df[col].str.strip().str.endswith('%', na=False).any()
    ]

    # If no percentage columns are found, return the original DataFrame
    if not percent_cols:
        print("No percentage columns found to modify.")  # Print message
        return df

    print("The following columns ending with % were modified:") #Print message before the loop

    # Process identified percentage columns
    for col in percent_cols:
        # Clean data: (1) Strip whitespace, (2) Handle '-', (3) Remove trailing %
        cleaned_series = (
            df[col].str.strip()
            .replace('-', np.nan)  # Convert '-' to NaN
            .str.replace(r'%$', '', regex=True)  # Remove only ENDING %
        )
        
        # Convert to numeric (coerce invalid values to NaN)
        df[col] = pd.to_numeric(cleaned_series, errors='coerce')
        
        # Rename column
        df.rename(columns={col: f"{col} %"}, inplace=True)
        print(f"- {col}")  # Print the column name
        
    return df

# Example Usage:
# Assuming you have a DataFrame called 'df'
# df = process_percentage_columns(df)

In [69]:
df = process_percentage_columns(df)

The following columns ending with % were modified:
- Dividend
- Perf Week
- Perf Month
- Perf Quart
- Perf Half
- Perf Year
- Perf YTD
- Volatility W
- Volatility M
- SMA20
- SMA50
- SMA200
- 50D High
- 50D Low
- 52W High
- 52W Low
- All-Time High
- All-Time Low
- Gap
- Change


In [70]:
# List of columns to concatenate
columns_to_concat = ["Sector", "Industry", "Single Category", "Asset Type"]

# Replace '-' with empty string in specified columns
for col in columns_to_concat:
    df[col] = df[col].replace('-', '')

# Concatenate the columns, handling empty strings, and remove extra spaces
df['Info'] = df[columns_to_concat].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)

In [71]:
# Concatenate the columns 'Market Cap' and 'AUM'
# Replace '-' with empty string in both columns

df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')

In [72]:
# Create new column by converting to numeric values in millions
df['MktCap AUM, M'] = df['MktCap AUM'].apply(convert_B_M_K_to_million)
df['Avg Volume, M'] = df['Avg Volume'].apply(convert_B_M_K_to_million)
df['Volume, M'] = (pd.to_numeric(df['Volume'].str.replace(',', ''), errors='coerce') / 1_000_000)

In [73]:
# Convert 'Beta', 'ATR', 'Rel Volume', and 'Price' to float
for col in ['Beta', 'ATR', 'RSI', 'Rel Volume', 'Price']:
  # Clean and convert to numeric
  df[col] = pd.to_numeric(
    df[col].str.replace('$', '').str.replace(',', ''),
    errors='coerce'  # Convert invalid values to NaN
  )

# Verify the conversion
for col in ['Beta', 'ATR', 'Rel Volume', 'Price']:
  print(f"{col}: {df[col].dtype}")

Beta: float64
ATR: float64
Rel Volume: float64
Price: float64


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   No.              1560 non-null   object 
 1   Ticker           1560 non-null   object 
 2   Company          1560 non-null   object 
 3   Sector           1560 non-null   object 
 4   Industry         1560 non-null   object 
 5   Market Cap       1560 non-null   object 
 6   Dividend %       1223 non-null   float64
 7   Perf Week %      1559 non-null   float64
 8   Perf Month %     1559 non-null   float64
 9   Perf Quart %     1555 non-null   float64
 10  Perf Half %      1553 non-null   float64
 11  Perf Year %      1544 non-null   float64
 12  Perf YTD %       1560 non-null   float64
 13  Beta             1542 non-null   float64
 14  ATR              1560 non-null   float64
 15  Volatility W %   1559 non-null   float64
 16  Volatility M %   1559 non-null   float64
 17  SMA20 %       

In [75]:
df.columns

Index(['No.', 'Ticker', 'Company', 'Sector', 'Industry', 'Market Cap',
       'Dividend %', 'Perf Week %', 'Perf Month %', 'Perf Quart %',
       'Perf Half %', 'Perf Year %', 'Perf YTD %', 'Beta', 'ATR',
       'Volatility W %', 'Volatility M %', 'SMA20 %', 'SMA50 %', 'SMA200 %',
       '50D High %', '50D Low %', '52W High %', '52W Low %', 'All-Time High %',
       'All-Time Low %', 'RSI', 'Gap %', 'Avg Volume', 'Rel Volume', 'Volume',
       'Price', 'Change %', 'Single Category', 'Asset Type', 'AUM', 'Info',
       'MktCap AUM', 'MktCap AUM, M', 'Avg Volume, M', 'Volume, M'],
      dtype='object')

In [76]:
my_cols = ['Ticker', 'Company', 'Info', 'MktCap AUM, M', 'Beta',
          'RSI', 'Perf YTD %', 'Perf Week %', 'Perf Month %', 'Perf Quart %', 'Perf Half %', 'Perf Year %',
          'SMA20 %', 'SMA50 %', 'SMA200 %',   
          '50D High %', '50D Low %', '52W High %', '52W Low %', 'All-Time High %', 'All-Time Low %',
          'ATR', 'Volatility W %', 'Volatility M %',  
          'Volume, M', 'Avg Volume, M','Rel Volume',
          'Price', 'Change %', 'Dividend %', 
          ]

In [77]:
# Create new df with my_cols and set Ticker as index
new_df = df[my_cols].set_index('Ticker')

# Sort by 'MktCap AUM, M' in descending order, with NaN values last
new_df = new_df.sort_values(by='MktCap AUM, M', ascending=False, na_position='last')

# Display info and first few rows to verify
display(new_df.info(), new_df)

<class 'pandas.core.frame.DataFrame'>
Index: 1560 entries, AAPL to FNDC
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1560 non-null   object 
 1   Info             1560 non-null   object 
 2   MktCap AUM, M    1560 non-null   float64
 3   Beta             1542 non-null   float64
 4   RSI              1559 non-null   float64
 5   Perf YTD %       1560 non-null   float64
 6   Perf Week %      1559 non-null   float64
 7   Perf Month %     1559 non-null   float64
 8   Perf Quart %     1555 non-null   float64
 9   Perf Half %      1553 non-null   float64
 10  Perf Year %      1544 non-null   float64
 11  SMA20 %          1560 non-null   float64
 12  SMA50 %          1560 non-null   float64
 13  SMA200 %         1560 non-null   float64
 14  50D High %       1560 non-null   float64
 15  50D Low %        1560 non-null   float64
 16  52W High %       1560 non-null   float64
 17  52W Low %       

None

,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,All-Time High %,All-Time Low %,ATR,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Dividend %
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",3352780.0,1.26,48.10,-10.87,-0.25,-6.23,-12.68,-2.02,30.16,0.85,-3.01,-2.60,-10.72,7.09,-14.19,36.03,-14.19,350739.37,6.03,2.68,2.76,36.350883,52.97000,0.69,223.19,0.48,0.47
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2841190.0,1.00,42.27,-9.33,-3.28,-1.62,-11.23,-10.71,-9.16,-1.32,-5.38,-9.42,-14.76,4.07,-18.40,4.07,-18.40,479564.32,8.77,2.30,2.19,19.063122,23.11000,0.82,382.19,1.81,0.86
NVDA,NVIDIA Corp,"Technology, Semiconductors",2687660.0,1.96,40.62,-17.98,-8.73,-3.43,-19.60,-9.27,21.91,-3.95,-11.08,-13.61,-26.06,6.27,-28.07,45.69,-28.07,330349.99,5.59,4.58,4.28,216.540419,281.46000,0.77,110.15,1.63,0.04
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",2036570.0,1.39,38.08,-12.41,-6.58,-6.27,-14.11,2.23,6.54,-2.75,-10.79,-4.03,-20.76,4.21,-20.76,26.75,-20.76,292730.49,6.49,3.30,3.08,41.136819,40.05000,1.03,192.17,1.00,NaN
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1925920.0,1.04,37.16,-16.57,-8.05,-5.80,-18.12,-3.88,4.35,-4.63,-11.80,-9.89,-23.87,4.38,-23.87,7.21,-23.87,553.52,5.03,3.41,2.87,20.079706,19.97000,1.01,158.88,1.70,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SPGP,Invesco S&P 500 GARP ETF,"Financial, Exchange Traded Fund, US Equities -...",3190.0,1.03,43.85,-4.39,-1.84,-2.33,-4.82,-4.33,-5.83,-0.04,-4.32,-4.45,-10.01,4.07,-11.17,5.54,-11.17,485.11,1.76,1.88,1.75,0.177552,0.26327,0.67,100.28,0.40,1.52
IBDQ,iShares iBonds Dec 2025 Term Corporate ETF,"Financial, Exchange Traded Fund, Bonds - Corpo...",3180.0,0.13,40.55,0.16,-0.24,0.04,0.24,-0.04,1.13,-0.12,-0.11,0.25,-0.28,0.08,-0.28,1.66,-10.33,11.58,0.02,0.05,0.06,0.523618,0.55480,0.94,25.09,-0.24,3.86
BKLC,BNY Mellon US Large Cap Core Equity ETF,"Financial, Exchange Traded Fund, US Equities -...",3180.0,1.03,40.92,-4.50,-3.01,-4.16,-5.97,-1.98,7.23,-0.93,-4.74,-2.38,-8.92,2.31,-8.92,13.96,-8.92,107.85,1.77,1.68,1.60,0.226461,0.11014,2.06,107.02,0.66,1.28


In [79]:
# Save to pickle file
# new_df.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
new_df.to_parquet(dest_path, engine='pyarrow')
print(f'save new_df to {dest_path}')

# # To load it later:
# loaded_df = pd.read_pickle('df_finviz.pkl')

save new_df to ..\data\df_finviz.parquet
